In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
%load_ext google.cloud.bigquery
from google.cloud import bigquery

client=bigquery.Client()
project_id = "som-nero-phi-jonc101"

/Users/sandychen/Desktop/Healthrex_workspace/scripts/Blood_Culture_Stewardship/blood_culture_env/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/sandychen/Desktop/Healthrex_workspace/scripts/Blood_Culture_Stewardship/blood_culture_env/lib/python3.9/site-packages/google/cloud/bigquery/__init__.py:237: FutureWarning: %load_ext google.cloud.bigquery is deprecated. Install bigquery-magics package and use `%load_ext bigquery_magics`, instead.
  warnings.warn(


# Data preprocessing mainly used to impute null values, scale parameters, and remove outliers

#### Load all data

In [ ]:
# demo
table_id_final_base_demo = f"{project_id}.blood_culture_stewardship_sandy_2024.final_base_bmi_age_race_gender_adi_temp"
final_base_demo = client.query(f"SELECT * FROM `{table_id_final_base_demo}`").to_dataframe()
# labs
table_id_final_base_labs_lda_ua = f"{project_id}.blood_culture_stewardship_sandy_2024.final_base_labs_without_cpr_or_pct_ua_lda"
final_base_labs_ua_lda= client.query(f"SELECT * FROM `{table_id_final_base_labs_lda_ua}`").to_dataframe()
# vitals
table_id_final_base_vitals = f"{project_id}.blood_culture_stewardship_sandy_2024.final_base_vitals"
final_base_vitals= client.query(f"SELECT * FROM `{table_id_final_base_vitals}`").to_dataframe()

table_id_final_labs_cr_only = f"{project_id}.blood_culture_stewardship_sandy_2024.final_base_labs_cr_only"
final_labs_cr_only = client.query(f"SELECT * FROM `{table_id_final_labs_cr_only}`").to_dataframe()



In [3]:
# Alternative: Using functools.reduce to merge multiple dataframes at once
from functools import reduce

# Define merge keys
key_cols = ["anon_id", "pat_enc_csn_id_coded", "order_proc_id_coded", "blood_culture_order_datetime_utc"]

# List of dataframes to merge
dfs_to_merge = [final_base_demo, final_base_labs_ua_lda, final_base_vitals, final_labs_cr_only]

# Merge all dataframes
all_features_df = reduce(
    lambda left, right: pd.merge(left, right, on=key_cols, how="inner"),
    dfs_to_merge
)

print(f"Shape of merged features: {all_features_df.shape}")
print(f"Number of rows: {len(all_features_df)}")
all_features_df.head()


Shape of merged features: (198015, 89)
Number of rows: 198015


,anon_id,pat_enc_csn_id_coded,order_proc_id_coded,blood_culture_order_datetime_utc,order_year,ed_arrival_datetime_utc,positive_blood_culture,gender,race,bmi,age,zip_clean,adi_score,adi_imputed_flag,min_wbc,max_wbc,avg_wbc,median_wbc,min_neutrophils,max_neutrophils,avg_neutrophils,median_neutrophils,min_anc,max_anc,avg_anc,median_anc,min_lymphocytes,max_lymphocytes,avg_lymphocytes,median_lymphocytes,min_alc,max_alc,avg_alc,median_alc,min_hgb,max_hgb,avg_hgb,median_hgb,min_plt,max_plt,avg_plt,median_plt,min_glucose,max_glucose,avg_glucose,median_glucose,min_lactate,max_lactate,avg_lactate,median_lactate,Leukocyte_Esterase,WBC_urine,Bacteria_urine,Nitrite_urine,has_any_line,temp_min_c,temp_avg_c,temp_max_c,temp_median_c,temp_mode_c,resp_min,resp_avg,resp_max,resp_median,resp_mode,hr_min,hr_avg,hr_max,hr_median,hr_mode,sysbp_min,sysbp_avg,sysbp_max,sysbp_median,sysbp_mode,diabp_min,diabp_avg,diabp_max,diabp_median,diabp_mode,spo2_min,spo2_avg,spo2_max,spo2_median,spo2_mode,min_cr,max_cr,avg_cr,median_cr
0,JC1483966,131019036862,401914892,2012-06-10 14:14:00,2012,2012-06-10 14:09:00+00:00,0,Female,Other,27.25,63,None,25.205909,1,9.2,9.2,9.2,9.2,NaN,NaN,NaN,NaN,7.14,7.14,7.14,7.14,13.7,13.7,13.7,13.7,1.26,1.26,1.26,1.26,10.0,10.0,10.0,10.0,507.0,507.0,507.0,507.0,83.0,94.0,88.5,83.0,0.47,0.47,0.47,0.47,None,None,None,None,0,36.388889,36.388889,36.388889,36.388889,36.4,20.0,22.0,24.0,20.0,20.0,72.0,73.5,75.0,72.0,72.0,135.0,162.0,189.0,135.0,135.0,93.0,93.5,94.0,93.0,93.0,89.0,94.0,99.0,89.0,89.0,1.00,1.10,1.03,1.00
1,JC3701091,131393811639,1014401137,2024-12-29 03:18:00,2024,2024-12-29 03:02:00+00:00,0,Female,Other,29.82,29,None,25.205909,1,9.0,9.0,9.0,9.0,72.2,72.2,72.2,72.2,6.52,6.52,6.52,6.52,18.6,18.6,18.6,18.6,1.68,1.68,1.68,1.68,12.7,12.7,12.7,12.7,471.0,471.0,471.0,471.0,94.0,94.0,94.0,94.0,0.70,0.70,0.70,0.70,NEGATIVE,NEGATIVE,POSITIVE,POSITIVE,0,39.055556,39.055556,39.055556,39.055556,39.1,16.0,20.5,25.0,16.0,16.0,115.0,121.5,128.0,115.0,115.0,112.0,118.0,124.0,112.0,112.0,64.0,69.0,74.0,64.0,64.0,98.0,98.5,99.0,98.0,98.0,0.62,0.62,0.62,0.62
2,JC3701091,131393811639,1014401136,2024-12-29 03:18:00,2024,2024-12-29 03:02:00+00:00,0,Female,Other,29.82,29,None,25.205909,1,9.0,9.0,9.0,9.0,72.2,72.2,72.2,72.2,6.52,6.52,6.52,6.52,18.6,18.6,18.6,18.6,1.68,1.68,1.68,1.68,12.7,12.7,12.7,12.7,471.0,471.0,471.0,471.0,94.0,94.0,94.0,94.0,0.70,0.70,0.70,0.70,NEGATIVE,NEGATIVE,POSITIVE,POSITIVE,0,39.055556,39.055556,39.055556,39.055556,39.1,16.0,20.5,25.0,16.0,16.0,115.0,121.5,128.0,115.0,115.0,112.0,118.0,124.0,112.0,112.0,64.0,69.0,74.0,64.0,64.0,98.0,98.5,99.0,98.0,98.0,0.62,0.62,0.62,0.62
3,JC3701091,131393980589,1015147480,2025-01-03 02:08:00,2025,2025-01-03 02:03:00+00:00,0,Female,Other,29.82,30,None,25.205909,1,7.6,7.6,7.6,7.6,62.8,62.8,62.8,62.8,4.75,4.75,4.75,4.75,26.3,26.3,26.3,26.3,1.99,1.99,1.99,1.99,11.6,11.6,11.6,11.6,487.0,487.0,487.0,487.0,111.0,111.0,111.0,111.0,0.70,0.70,0.70,0.70,None,None,None,None,0,36.833333,36.916667,37.000000,36.833333,36.8,18.0,24.5,31.0,18.0,18.0,100.0,110.0,120.0,100.0,100.0,102.0,113.5,125.0,102.0,102.0,65.0,70.5,76.0,65.0,65.0,98.0,99.0,100.0,98.0,98.0,0.60,0.60,0.60,0.60
4,JC3701091,131393980589,1015147481,2025-01-03 02:08:00,2025,2025-01-03 02:03:00+00:00,0,Female,Other,29.82,30,None,25.205909,1,7.6,7.6,7.6,7.6,62.8,62.8,62.8,62.8,4.75,4.75,4.75,4.75,26.3,26.3,26.3,26.3,1.99,1.99,1.99,1.99,11.6,11.6,11.6,11.6,487.0,487.0,487.0,487.0,111.0,111.0,111.0,111.0,0.70,0.70,0.70,0.70,None,None,None,None,0,36.833333,36.916667,37.000000,36.833333,36.8,18.0,24.5,31.0,18.0,18.0,100.0,110.0,120.0,100.0,100.0,102.0,113.5,125.0,102.0,102.0,65.0,70.5,76.0,65.0,65.0,98.0,99.0,100.0,98.0,98.0,0.60,0.60,0.60,0.60


In [9]:
all_features_df

,anon_id,pat_enc_csn_id_coded,order_proc_id_coded,blood_culture_order_datetime_utc,order_year,ed_arrival_datetime_utc,positive_blood_culture,gender,race,bmi,age,zip_clean,adi_score,adi_imputed_flag,min_wbc,max_wbc,avg_wbc,median_wbc,min_neutrophils,max_neutrophils,avg_neutrophils,median_neutrophils,min_anc,max_anc,avg_anc,median_anc,min_lymphocytes,max_lymphocytes,avg_lymphocytes,median_lymphocytes,min_alc,max_alc,avg_alc,median_alc,min_hgb,max_hgb,avg_hgb,median_hgb,min_plt,max_plt,avg_plt,median_plt,min_glucose,max_glucose,avg_glucose,median_glucose,min_lactate,max_lactate,avg_lactate,median_lactate,Leukocyte_Esterase,WBC_urine,Bacteria_urine,Nitrite_urine,has_any_line,temp_min_c,temp_avg_c,temp_max_c,temp_median_c,temp_mode_c,resp_min,resp_avg,resp_max,resp_median,resp_mode,hr_min,hr_avg,hr_max,hr_median,hr_mode,sysbp_min,sysbp_avg,sysbp_max,sysbp_median,sysbp_mode,diabp_min,diabp_avg,diabp_max,diabp_median,diabp_mode,spo2_min,spo2_avg,spo2_max,spo2_median,spo2_mode,min_cr,max_cr,avg_cr,median_cr
0,JC1483966,131019036862,401914892,2012-06-10 14:14:00,2012,2012-06-10 14:09:00+00:00,0,Female,Other,27.25,63,None,25.205909,1,9.2,9.2,9.20,9.2,NaN,NaN,NaN,NaN,7.14,7.14,7.14,7.14,13.7,13.7,13.7,13.7,1.26,1.26,1.26,1.26,10.0,10.0,10.00,10.0,507.0,507.0,507.0,507.0,83.0,94.0,88.50,83.0,0.47,0.47,0.47,0.47,None,None,None,None,0,36.388889,36.388889,36.388889,36.388889,36.4,20.0,22.000000,24.0,20.0,20.0,72.0,73.500000,75.0,72.0,72.0,135.0,162.000000,189.0,135.0,135.0,93.0,93.500000,94.0,93.0,93.0,89.0,94.000000,99.0,89.0,89.0,1.00,1.10,1.03,1.00
1,JC3701091,131393811639,1014401137,2024-12-29 03:18:00,2024,2024-12-29 03:02:00+00:00,0,Female,Other,29.82,29,None,25.205909,1,9.0,9.0,9.00,9.0,72.2,72.2,72.20,72.2,6.52,6.52,6.52,6.52,18.6,18.6,18.6,18.6,1.68,1.68,1.68,1.68,12.7,12.7,12.70,12.7,471.0,471.0,471.0,471.0,94.0,94.0,94.00,94.0,0.70,0.70,0.70,0.70,NEGATIVE,NEGATIVE,POSITIVE,POSITIVE,0,39.055556,39.055556,39.055556,39.055556,39.1,16.0,20.500000,25.0,16.0,16.0,115.0,121.500000,128.0,115.0,115.0,112.0,118.000000,124.0,112.0,112.0,64.0,69.000000,74.0,64.0,64.0,98.0,98.500000,99.0,98.0,98.0,0.62,0.62,0.62,0.62
2,JC3701091,131393811639,1014401136,2024-12-29 03:18:00,2024,2024-12-29 03:02:00+00:00,0,Female,Other,29.82,29,None,25.205909,1,9.0,9.0,9.00,9.0,72.2,72.2,72.20,72.2,6.52,6.52,6.52,6.52,18.6,18.6,18.6,18.6,1.68,1.68,1.68,1.68,12.7,12.7,12.70,12.7,471.0,471.0,471.0,471.0,94.0,94.0,94.00,94.0,0.70,0.70,0.70,0.70,NEGATIVE,NEGATIVE,POSITIVE,POSITIVE,0,39.055556,39.055556,39.055556,39.055556,39.1,16.0,20.500000,25.0,16.0,16.0,115.0,121.500000,128.0,115.0,115.0,112.0,118.000000,124.0,112.0,112.0,64.0,69.000000,74.0,64.0,64.0,98.0,98.500000,99.0,98.0,98.0,0.62,0.62,0.62,0.62
3,JC3701091,131393980589,1015147480,2025-01-03 02:08:00,2025,2025-01-03 02:03:00+00:00,0,Female,Other,29.82,30,None,25.205909,1,7.6,7.6,7.60,7.6,62.8,62.8,62.80,62.8,4.75,4.75,4.75,4.75,26.3,26.3,26.3,26.3,1.99,1.99,1.99,1.99,11.6,11.6,11.60,11.6,487.0,487.0,487.0,487.0,111.0,111.0,111.00,111.0,0.70,0.70,0.70,0.70,None,None,None,None,0,36.833333,36.916667,37.000000,36.833333,36.8,18.0,24.500000,31.0,18.0,18.0,100.0,110.000000,120.0,100.0,100.0,102.0,113.500000,125.0,102.0,102.0,65.0,70.500000,76.0,65.0,65.0,98.0,99.000000,100.0,98.0,98.0,0.60,0.60,0.60,0.60
4,JC3701091,131393980589,1015147481,2025-01-03 02:08:00,2025,2025-01-03 02:03:00+00:00,0,Female,Other,29.82,30,None,25.205909,1,7.6,7.6,7.60,7.6,62.8,62.8,62.80,62.8,4.75,4.75,4.75,4.75,26.3,26.3,26.3,26.3,1.99,1.99,1.99,1.99,11.6,11.6,11.60,11.6,487.0,487.0,487.0,487.0,111.0,111.0,111.00,111.0,0.70,0.70,0.70,0.70,None,None,None,None,0,36.833333,36.916667,37.000000,36.833333,36.8,18.0,24.500000,31.0,18.0,18.0,100.0,110.000000,120.0,100.0,100.0,102.0,113.500000,125.0,102.0,102.0,65.0,70.500000,76.0,65.0,65.0,98.0,99.000000,100.0,98.0,98.0,0.60,0.60,0.60,0.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [4]:
all_features_df_dedup = all_features_df.drop_duplicates(subset = ["anon_id", "pat_enc_csn_id_coded"])

In [8]:
all_features_df_dedup.columns

Index(['anon_id', 'pat_enc_csn_id_coded', 'order_proc_id_coded',
       'blood_culture_order_datetime_utc', 'order_year',
       'ed_arrival_datetime_utc', 'positive_blood_culture', 'gender', 'race',
       'bmi', 'age', 'zip_clean', 'adi_score', 'adi_imputed_flag', 'min_wbc',
       'max_wbc', 'avg_wbc', 'median_wbc', 'min_neutrophils',
       'max_neutrophils', 'avg_neutrophils', 'median_neutrophils', 'min_anc',
       'max_anc', 'avg_anc', 'median_anc', 'min_lymphocytes',
       'max_lymphocytes', 'avg_lymphocytes', 'median_lymphocytes', 'min_alc',
       'max_alc', 'avg_alc', 'median_alc', 'min_hgb', 'max_hgb', 'avg_hgb',
       'median_hgb', 'min_plt', 'max_plt', 'avg_plt', 'median_plt',
       'min_glucose', 'max_glucose', 'avg_glucose', 'median_glucose',
       'min_lactate', 'max_lactate', 'avg_lactate', 'median_lactate',
       'Leukocyte_Esterase', 'WBC_urine', 'Bacteria_urine', 'Nitrite_urine',
       'has_any_line', 'temp_min_c', 'temp_avg_c', 'temp_max_c',
       'temp_med

## data split

### split into train, val, test (with ~70%,~15%,~15%)
### use longitudinal split
### train: start - 2023 (Exclusive)
### val: 2023-2024 (exclusive)
### test: 2024-now (inclusive)
### the prevalence is about 5% for all data split

In [5]:

print(f"total number of rows: {all_features_df.shape[0]}")
train_data = all_features_df[all_features_df['order_year'] < 2023]
val_data = all_features_df[(all_features_df['order_year'] >= 2023) & (all_features_df['order_year'] < 2024)]
test_data = all_features_df[all_features_df['order_year'] >= 2024]

print(f"Train data shape: {train_data.shape} and is {train_data.shape[0] / all_features_df.shape[0] * 100}% of all data")
print(f"Val data shape: {val_data.shape} and is {val_data.shape[0] / all_features_df.shape[0] * 100}% of all data")
print(f"Test data shape: {test_data.shape} and is {test_data.shape[0] / all_features_df.shape[0] * 100}% of all data")

total number of rows: 198015
Train data shape: (141286, 89) and is 71.35116026563644% of all data
Val data shape: (26606, 89) and is 13.436355831628918% of all data
Test data shape: (30123, 89) and is 15.212483902734641% of all data


In [6]:
# prevalence of blood culture positive
print(f"prevalence of blood culture positive in all data: {all_features_df['positive_blood_culture'].mean() *100}%")
print(f"prevalence of blood culture positive in train data: {train_data['positive_blood_culture'].mean() *100}%")
print(f"prevalence of blood culture positive in val data: {val_data['positive_blood_culture'].mean() *100}%")
print(f"prevalence of blood culture positive in test data: {test_data['positive_blood_culture'].mean() *100}%")

prevalence of blood culture positive in all data: 7.728707421154963%
prevalence of blood culture positive in train data: 8.045383123593279%
prevalence of blood culture positive in val data: 6.7503570623167715%
prevalence of blood culture positive in test data: 7.107525810842214%


In [7]:
print(f"total number of rows after dedup: {all_features_df_dedup.shape[0]}")
train_data_dedup = all_features_df_dedup[all_features_df_dedup['order_year'] < 2023]
val_data_dedup = all_features_df_dedup[(all_features_df_dedup['order_year'] >= 2023) & (all_features_df_dedup['order_year'] < 2024)]
test_data_dedup = all_features_df_dedup[all_features_df_dedup['order_year'] >= 2024]

print(f"Train data shape: {train_data_dedup.shape} and is {train_data_dedup.shape[0] / all_features_df_dedup.shape[0] * 100}% of all data")
print(f"Val data shape: {val_data_dedup.shape} and is {val_data_dedup.shape[0] / all_features_df_dedup.shape[0] * 100}% of all data")
print(f"Test data shape: {test_data_dedup.shape} and is {test_data_dedup.shape[0] / all_features_df_dedup.shape[0] * 100}% of all data")

total number of rows after dedup: 102015
Train data shape: (73393, 89) and is 71.94334166544135% of all data
Val data shape: (13441, 89) and is 13.175513404891436% of all data
Test data shape: (15181, 89) and is 14.881144929667206% of all data


In [8]:
# prevalence of blood culture positive
print(f"prevalence of blood culture positive in all data: {all_features_df_dedup['positive_blood_culture'].mean() *100}%")
print(f"prevalence of blood culture positive in train data_dedup: {train_data_dedup['positive_blood_culture'].mean() *100}%")
print(f"prevalence of blood culture positive in val data_dedup: {val_data_dedup['positive_blood_culture'].mean() *100}%")
print(f"prevalence of blood culture positive in test data_dedup: {test_data_dedup['positive_blood_culture'].mean() *100}%")

prevalence of blood culture positive in all data: 7.714551781600745%
prevalence of blood culture positive in train data_dedup: 8.026651043015%
prevalence of blood culture positive in val data_dedup: 6.733129975448255%
prevalence of blood culture positive in test data_dedup: 7.074632764640011%


### process demo data
##### drop unknown gender and impute BMI by age and gender

In [11]:
import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
import sklearn

# ===============================
# 0) Helpers
# ===============================
def fill_demo_unknown(df):
    out = df.copy()
    if "gender" in out.columns:
        out["gender"] = out["gender"].fillna("Unknown")
    if "race" in out.columns:
        out["race"] = out["race"].fillna("Unknown")
    return out

def make_age_group(df, age_col="age"):
    bins   = [18, 40, 65, 200]  # adult-only
    labels = ["young_adult", "middle_age", "older_adult"]
    out = df.copy()
    out["age_group"] = pd.cut(out[age_col], bins=bins, labels=labels, right=False)
    return out

def fit_group_medians(train_df, group_cols, numeric_cols):
    gm = (train_df.groupby(group_cols, dropna=False)[numeric_cols]
                  .median()
                  .reset_index())
    global_medians = train_df[numeric_cols].median()
    return gm, global_medians

def apply_stratified_impute(df, group_cols, group_medians, global_medians,
                            numeric_cols, ua_defaults):
    out = df.copy()
    out = out.merge(group_medians, on=group_cols, how="left", suffixes=("", "__grpmed"))

    # numeric columns: per-group median -> global median
    for col in numeric_cols:
        if col not in out.columns:
            continue
        out[f"{col}_missing_flag"] = out[col].isna().astype(int)
        grp_col = f"{col}__grpmed"
        out[col] = out[col].fillna(out.get(grp_col))
        if out[col].isna().any():
            out[col] = out[col].fillna(global_medians.get(col))
        if grp_col in out.columns:
            out.drop(columns=[grp_col], inplace=True)

    # UA + binary defaults
    for col, default_val in ua_defaults.items():
        if col in out.columns:
            out[f"{col}_missing_flag"] = out[col].isna().astype(int)
            out[col] = out[col].fillna(default_val)
            if col == "has_any_line":
                try:
                    out[col] = out[col].astype("Int64")
                except Exception:
                    pass
    return out

def check_missing(df, cols, name):
    miss = df[cols].isna().mean().sort_values(ascending=False).head(20)
    print(f"[{name}] top remaining missing (should be 0):\n{miss}\n")

# ===============================
# 1) Fill demo Unknown + make age_group
# ===============================
for name in ("train_data", "val_data", "test_data"):
    locals()[name] = fill_demo_unknown(locals()[name])
    locals()[name] = make_age_group(locals()[name], age_col="age")

# ===============================
# 2) Feature selection (ALL vitals + ALL lab aggregates)
# ===============================
label_key = "positive_blood_culture"
demo_cols = ["gender", "race", "age", "bmi", "adi_score"]

vital_prefixes = ("temp_", "hr_", "sysbp_", "diabp_", "spo2_", "resp_")
lab_bases      = ("wbc","neutrophils","anc","lymphocytes","alc","hgb","plt","glucose","lactate", "cr")
lab_agg_prefix = ("min_", "max_", "avg_", "median_", "mode_")  # mode_* may not exist for all

ua_cols_all    = ["Leukocyte_Esterase","WBC_urine","Bacteria_urine","Nitrite_urine","has_any_line"]

all_cols = set(train_data.columns)

# vitals present
vital_cols = [c for c in all_cols if c.startswith(vital_prefixes)]

# lab aggregates present (min/max/avg/median/mode for chosen lab families)
lab_agg_cols = [
    c for c in all_cols
    if any(c.startswith(pref) for pref in lab_agg_prefix)
    and any(c.endswith(f"_{base}") for base in lab_bases)
]

# UA present
ua_cols_present = [c for c in ua_cols_all if c in all_cols]

# Final feature list used in modeling
selected_features = [c for c in (demo_cols + vital_cols + lab_agg_cols + ua_cols_present) if c in all_cols]

# Numeric targets for stratified imputation (vitals + all lab aggs + numeric demos)
numeric_targets = [
    c for c in (list(vital_cols) + list(lab_agg_cols) + ["age","bmi","adi_score"])
    if c in train_data.columns and pd.api.types.is_numeric_dtype(train_data[c])
]

# UA defaults for missing (categorical dipsticks as NEGATIVE, line as 0)
ua_defaults = {
    "Leukocyte_Esterase": "NEGATIVE",
    "WBC_urine": "NEGATIVE",
    "Bacteria_urine": "NEGATIVE",
    "Nitrite_urine": "NEGATIVE",
    "has_any_line": 0,   # keep as binary
}

# ===============================
# 3) Train-only stratified imputation by (gender × age_group)
# ===============================
group_cols = ["gender", "age_group"]
group_meds, global_meds = fit_group_medians(train_data, group_cols, numeric_targets)

train_imp = apply_stratified_impute(train_data, group_cols, group_meds, global_meds,
                                    numeric_targets, ua_defaults)
val_imp   = apply_stratified_impute(val_data,   group_cols, group_meds, global_meds,
                                    numeric_targets, ua_defaults)
test_imp  = apply_stratified_impute(test_data,  group_cols, group_meds, global_meds,
                                    numeric_targets, ua_defaults)

# Sanity checks (only on columns we actually use)
check_cols = [c for c in (numeric_targets + ua_cols_present) if c in train_imp.columns]
check_missing(train_imp, check_cols, "train")
check_missing(val_imp,   check_cols, "val")
check_missing(test_imp,  check_cols, "test")

/var/folders/d1/3gdyy98d6h1d9pxx47s40vv40000gp/T/ipykernel_13373/51115027.py:26: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  gm = (train_df.groupby(group_cols, dropna=False)[numeric_cols]
/var/folders/d1/3gdyy98d6h1d9pxx47s40vv40000gp/T/ipykernel_13373/51115027.py:41: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  out[f"{col}_missing_flag"] = out[col].isna().astype(int)
/var/folders/d1/3gdyy98d6h1d9pxx47s40vv40000gp/T/ipykernel_13373/51115027.py:41: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, wh

[train] top remaining missing (should be 0):
hr_mode         0.0
hr_avg          0.0
diabp_max       0.0
sysbp_avg       0.0
spo2_max        0.0
diabp_mode      0.0
spo2_mode       0.0
resp_mode       0.0
resp_min        0.0
spo2_min        0.0
hr_median       0.0
resp_max        0.0
spo2_median     0.0
spo2_avg        0.0
temp_min_c      0.0
diabp_min       0.0
sysbp_min       0.0
diabp_median    0.0
sysbp_max       0.0
diabp_avg       0.0
dtype: float64

[val] top remaining missing (should be 0):
hr_mode         0.0
hr_avg          0.0
diabp_max       0.0
sysbp_avg       0.0
spo2_max        0.0
diabp_mode      0.0
spo2_mode       0.0
resp_mode       0.0
resp_min        0.0
spo2_min        0.0
hr_median       0.0
resp_max        0.0
spo2_median     0.0
spo2_avg        0.0
temp_min_c      0.0
diabp_min       0.0
sysbp_min       0.0
diabp_median    0.0
sysbp_max       0.0
diabp_avg       0.0
dtype: float64

[test] top remaining missing (should be 0):
hr_mode         0.0
hr_avg          

In [9]:
table_id_all_features_imputed_train= f"{project_id}.blood_culture_stewardship_sandy_2024.final_base_features_imputed_train"
table_id_all_features_imputed_val= f"{project_id}.blood_culture_stewardship_sandy_2024.final_base_features_imputed_val"
table_id_all_features_imputed_test= f"{project_id}.blood_culture_stewardship_sandy_2024.final_base_features_imputed_test"

# Upload the DataFrame to BigQuery
train_imp.to_gbq(
    destination_table=table_id_all_features_imputed_train,
    project_id=project_id,
    if_exists='replace'  # This will replace the table if it exists
)
val_imp.to_gbq(
    destination_table=table_id_all_features_imputed_val,
    project_id=project_id,
    if_exists='replace'  # This will replace the table if it exists
)
test_imp.to_gbq(
    destination_table=table_id_all_features_imputed_test,
    project_id=project_id,
    if_exists='replace'  # This will replace the table if it exists
)
print(f"Uploaded train_imp to {table_id_all_features_imputed_train}")
print(f"Uploaded val_imp to {table_id_all_features_imputed_val}")
print(f"Uploaded test_imp to {table_id_all_features_imputed_test}")


/var/folders/d1/3gdyy98d6h1d9pxx47s40vv40000gp/T/ipykernel_8991/1841804406.py:6: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  train_imp.to_gbq(
100%|██████████| 1/1 [00:00<00:00, 1979.38it/s]
/var/folders/d1/3gdyy98d6h1d9pxx47s40vv40000gp/T/ipykernel_8991/1841804406.py:11: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  val_imp.to_gbq(
100%|██████████| 1/1 [00:00<00:00, 2582.70it/s]
/var/folders/d1/3gdyy98d6h1d9pxx47s40vv40000gp/T/ipykernel_8991/1841804406.py:16: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  test_imp.to_gbq(
100%|██████████| 1/1 [00:00<00:00, 10512.04i

Uploaded train_imp to som-nero-phi-jonc101.blood_culture_stewardship_sandy_2024.final_base_features_imputed_train
Uploaded val_imp to som-nero-phi-jonc101.blood_culture_stewardship_sandy_2024.final_base_features_imputed_val
Uploaded test_imp to som-nero-phi-jonc101.blood_culture_stewardship_sandy_2024.final_base_features_imputed_test


### imputed train,val test data uploaded to bigquery 
##### Uploaded train_imp to som-nero-phi-jonc101.blood_culture_stewardship_sandy_2024.final_base_features_imputed_train
##### Uploaded val_imp to som-nero-phi-jonc101.blood_culture_stewardship_sandy_2024.final_base_features_imputed_val
##### Uploaded test_imp to som-nero-phi-jonc101.blood_culture_stewardship_sandy_2024.final_base_features_imputed_test

# Dedup data

In [13]:
for name in ("train_data_dedup", "val_data_dedup", "test_data_dedup"):
    locals()[name] = fill_demo_unknown(locals()[name])
    locals()[name] = make_age_group(locals()[name], age_col="age")
group_meds, global_meds = fit_group_medians(train_data_dedup, group_cols, numeric_targets)

train_imp_dedup = apply_stratified_impute(train_data_dedup, group_cols, group_meds, global_meds,
                                    numeric_targets, ua_defaults)
val_imp_dedup   = apply_stratified_impute(val_data_dedup,   group_cols, group_meds, global_meds,
                                    numeric_targets, ua_defaults)
test_imp_dedup  = apply_stratified_impute(test_data_dedup,  group_cols, group_meds, global_meds,
                                    numeric_targets, ua_defaults)

# Sanity checks (only on columns we actually use)
check_cols = [c for c in (numeric_targets + ua_cols_present) if c in train_imp_dedup.columns]
check_missing(train_imp_dedup, check_cols, "train_dedup")
check_missing(val_imp_dedup,   check_cols, "val_dedup")
check_missing(test_imp_dedup,  check_cols, "test_dedup")

/var/folders/d1/3gdyy98d6h1d9pxx47s40vv40000gp/T/ipykernel_13373/51115027.py:26: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  gm = (train_df.groupby(group_cols, dropna=False)[numeric_cols]
/var/folders/d1/3gdyy98d6h1d9pxx47s40vv40000gp/T/ipykernel_13373/51115027.py:41: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  out[f"{col}_missing_flag"] = out[col].isna().astype(int)
/var/folders/d1/3gdyy98d6h1d9pxx47s40vv40000gp/T/ipykernel_13373/51115027.py:41: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, wh

[train_dedup] top remaining missing (should be 0):
hr_mode         0.0
hr_avg          0.0
diabp_max       0.0
sysbp_avg       0.0
spo2_max        0.0
diabp_mode      0.0
spo2_mode       0.0
resp_mode       0.0
resp_min        0.0
spo2_min        0.0
hr_median       0.0
resp_max        0.0
spo2_median     0.0
spo2_avg        0.0
temp_min_c      0.0
diabp_min       0.0
sysbp_min       0.0
diabp_median    0.0
sysbp_max       0.0
diabp_avg       0.0
dtype: float64

[val_dedup] top remaining missing (should be 0):
hr_mode         0.0
hr_avg          0.0
diabp_max       0.0
sysbp_avg       0.0
spo2_max        0.0
diabp_mode      0.0
spo2_mode       0.0
resp_mode       0.0
resp_min        0.0
spo2_min        0.0
hr_median       0.0
resp_max        0.0
spo2_median     0.0
spo2_avg        0.0
temp_min_c      0.0
diabp_min       0.0
sysbp_min       0.0
diabp_median    0.0
sysbp_max       0.0
diabp_avg       0.0
dtype: float64

[test_dedup] top remaining missing (should be 0):
hr_mode         0.

In [15]:
table_id_all_features_imputed_train_dedup= f"{project_id}.blood_culture_stewardship_sandy_2024.final_base_features_imputed_train_dedup"
table_id_all_features_imputed_val_dedup= f"{project_id}.blood_culture_stewardship_sandy_2024.final_base_features_imputed_val_dedup"
table_id_all_features_imputed_test_dedup= f"{project_id}.blood_culture_stewardship_sandy_2024.final_base_features_imputed_test_dedup"

# Upload the DataFrame to BigQuery
train_imp_dedup.to_gbq(
    destination_table=table_id_all_features_imputed_train_dedup,
    project_id=project_id,
    if_exists='replace'  # This will replace the table if it exists
)
val_imp_dedup.to_gbq(
    destination_table=table_id_all_features_imputed_val_dedup,
    project_id=project_id,
    if_exists='replace'  # This will replace the table if it exists
)
test_imp_dedup.to_gbq(
    destination_table=table_id_all_features_imputed_test_dedup,
    project_id=project_id,
    if_exists='replace'  # This will replace the table if it exists
)
print(f"Uploaded train_imp_dedup to {table_id_all_features_imputed_train_dedup}")
print(f"Uploaded val_imp_dedup to {table_id_all_features_imputed_val_dedup}")
print(f"Uploaded test_imp_dedup to {table_id_all_features_imputed_test_dedup}")


/var/folders/d1/3gdyy98d6h1d9pxx47s40vv40000gp/T/ipykernel_13373/2365035709.py:6: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  train_imp_dedup.to_gbq(
100%|██████████| 1/1 [00:00<00:00, 1140.38it/s]
/var/folders/d1/3gdyy98d6h1d9pxx47s40vv40000gp/T/ipykernel_13373/2365035709.py:11: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  val_imp_dedup.to_gbq(
100%|██████████| 1/1 [00:00<00:00, 1652.60it/s]
/var/folders/d1/3gdyy98d6h1d9pxx47s40vv40000gp/T/ipykernel_13373/2365035709.py:16: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  test_imp_dedup.to_gbq(
100%|██████████| 1/1 [0

Uploaded train_imp_dedup to som-nero-phi-jonc101.blood_culture_stewardship_sandy_2024.final_base_features_imputed_train_dedup
Uploaded val_imp_dedup to som-nero-phi-jonc101.blood_culture_stewardship_sandy_2024.final_base_features_imputed_val_dedup
Uploaded test_imp_dedup to som-nero-phi-jonc101.blood_culture_stewardship_sandy_2024.final_base_features_imputed_test_dedup
